## Write Lyrics

we are going to write songs based on previous data.

[Link for Dataset](https://www.kaggle.com/mousehead/songlyrics)

In [1]:
# import necessary libraries
import sys
import numpy as np
import pandas as pd
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\Users\z003w00f\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\z003w00f\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np

In [2]:
# loading the dataset
filename="lyrics.txt"
text= open(filename, 'r', encoding='utf-8').read()
text=text.lower()


In [3]:
# Create a sorted list of the characters
chars = sorted(list(set(text)))
print('total chars:', len(chars))

total chars: 68


In [4]:
# Output the length of the corpus
print('length:', len(text))

length: 260342


In [5]:
# training for first 8000000 text 
text = text[:100000]
print('length:', len(text))

length: 100000


In [6]:
# create mapping of all unique chars to integers
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
n_chars = len(text)
n_vocabs = len(chars)
print("Total characters: ", n_chars)
print("Total vocab: ", n_vocabs)

Total characters:  100000
Total vocab:  68


In [8]:
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length , 1):
    seq_in = text[i:i + seq_length]
    seq_out =text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append([char_to_int[seq_out]])

n_patterns = len(dataX)
print("Total patterns: ", n_patterns)

Total patterns:  99900


since LSTMs accept values in the form (no_of_sampels, time_steps, no_of_features), therefore
reshape dataX to this form

In [9]:
# reshape dataX
X = np.reshape(dataX, (n_patterns, seq_length, 1))

# normalize
X = X/float(n_vocabs)

# one hot encoding using np_utils
y = np_utils.to_categorical(dataY)

In [10]:
X.shape

(99900, 100, 1)

In [11]:
y.shape

(99900, 67)

In [12]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(rate=0.1))
model.add(LSTM(256))
model.add(Dropout(rate=0.1))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [13]:
# define the checkpoints and callbacks
filepath="SavedModel/weights-imporvement-{epoch: 02d}-{loss: .4f}-from-class.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [14]:
filename = "SavedModel/weights-imporvement- 50- 1.3979-from-class.hdf5" # add the name of your best trained saved model's name here
model.load_weights(filename)

In [28]:
# training the LSTM


# model.fit(X, y, epochs=150, batch_size=64, callbacks=callbacks_list,initial_epoch=50)

Epoch 51/150
99900/99900 [==============================] - 1145s 11ms/step - loss: 1.3875

Epoch 00051: loss improved from 1.39788 to 1.38745, saving model to SavedModel/weights-imporvement- 51- 1.3875-from-class.hdf5
Epoch 52/150
99900/99900 [==============================] - 1117s 11ms/step - loss: 1.3830

Epoch 00052: loss improved from 1.38745 to 1.38296, saving model to SavedModel/weights-imporvement- 52- 1.3830-from-class.hdf5
Epoch 53/150
99900/99900 [==============================] - 1124s 11ms/step - loss: 1.3725

Epoch 00053: loss improved from 1.38296 to 1.37247, saving model to SavedModel/weights-imporvement- 53- 1.3725-from-class.hdf5
Epoch 54/150
99900/99900 [==============================] - 2891s 29ms/step - loss: 1.3643

Epoch 00054: loss improved from 1.37247 to 1.36431, saving model to SavedModel/weights-imporvement- 54- 1.3643-from-class.hdf5
Epoch 55/150
99900/99900 [==============================] - 897s 9ms/step - loss: 1.3635

Epoch 00055: loss improved from 1.

KeyboardInterrupt: 

In [15]:
# Generating Text from pretrained/post training your LSTM

filename = "SavedModel/weights-imporvement- 125- 1.1999-from-class.hdf5" # add the name of your best trained saved model's name here
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [21]:
# set up a random seed for starting
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]

print("INPUT SEED:")
print("\"", ''.join([int_to_char[val] for val in pattern]), "\"")
print()
# generate characters from the generated output of LSTM
for i in range(700):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x/float(n_vocabs)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1: len(pattern)]
print("\nTHE END.")

INPUT SEED:
" on a minute, ah!
do anybody feel bad for bill cosby?
did he forget the names just like steve harvey? "

 (yeezy, yeezy, yeezy, i meed to stop again
bacy gor my friek, rurped
to do the god of the tame thing i know it's no jnde when i get it
we goin' on my bickc
in the damn mames
we roow this doun shgt'a whlle pict the ducking that i hot a baw berter clame me what i don't tell the tame thing
i know it's no puac
lasgie' well so mige
sooe like you can't say this for a mew friend and ii you got a man with a bitch
is all off in a lomer but i'm on the ptesenc
i'm foing on the nast we was up well i got the way that she doccr on the fay from the carkroiny
oow she got me on the tame thing
i know it's no jnre, the was fron the wale i wake up and i can't get to and sood a thggt, rhete niggas that lise tha
THE END.
